# Myers Briggs Personality Type Classification - Full

### Carolina Analytics and Data Science

Data from [Kaggle](https://www.kaggle.com/datasnaek/mbti-type)

The goal for this dataset will to be able to classify Myers Briggs Personality Type (MBTI) based on text someone with that personality type had written. In our meeting on Febuary 15 we will choose which 

This is starter code to get working on this dataset.

Note there are two versions of this document. In this (the "abridged") we start with already featurized text that we have made here so that jumping right into the modelling is also an option. 

In [1]:
import tensorflow as tf # neural net library
import numpy as np      # numerical/linear algebra
import pandas as pd     # reading in data
import re               # regular expression
import wordcloud
import csv

Here is how we output these to csvs which is what is used for the abridged version.

In [23]:
full_training_featurized = pd.read_csv(featurized_train.csv')
featurized_testing = pd.read_csv('featurized_test.csv')

## Prediction

Splitting into test and validation sets

In [24]:
full_training_featurized['Y'] = (1*(full_training_featurized['ie'] == 'I')) # having a Y column that is 0-1 valued

np.random.seed(seed=1) # for consistency with test and training sets
training_data, validation_data = np.split(full_training_featurized.sample(frac=1), [int(.75*len(full_training_featurized))]) # splitting into training and validation sets

### Logistic Regression

In [25]:
training_x = training_data.drop(['ie', 'sn', 'tf', 'pj', 'Y'], axis=1)
training_y = training_data['Y']

validation_x = validation_data.drop(['ie', 'sn', 'tf', 'pj', 'Y'], axis=1)
validation_y = validation_data['Y']

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg = model.fit(training_x, training_y)

logreg.score(training_x, training_y) # training accuracy

NameError: name 'model' is not defined

In [ ]:
def evaluate_model(model, validation_x, validation_y): 
    # testing accuracy 
    correct = 0
    for (a, b) in zip(model.predict(validation_x), validation_y):
        correct += (a-b)**2
    return float(correct)/len(validation_y)

evaluate_model(logreg, validation_x, validation_y)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest = random_forest.fit(training_x, training_y)

random_forest.score(training_x, training_y) # training accuracy

In [ ]:
evaluate_model(random_forest, validation_x, validation_y) # testing accuracy 

### Tensorflow Neural Net

In [ ]:
tf_training_y = np.array([training_y, -(training_y-1)]).T
tf_validation_y = np.array([validation_y, -(validation_y-1)]).T
# Parameters
learning_rate = 0.001
training_epochs = 1000
batch_size = 100
display_step = 1


# Network Parameters
n_hidden_1 = 50 # 1st layer number of features
n_hidden_2 = 50 # 2nd layer number of features
n_input = 50 # Number of feature
n_classes = 2 # Number of classes to predict

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(training_x)/batch_size)
        X_batches = np.array_split(training_x, total_batch)
        Y_batches = np.array_split(tf_training_y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Testing Accuracy:", accuracy.eval({x: validation_x, y: tf_validation_y}))
    global result 
    result = tf.argmax(pred, 1).eval({x: validation_x, y: tf_validation_y})

In [29]:
featurized_testing

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
instance,,,,,,,,,,,,,,,,,,,,,
2,0.005180,0.000332,-0.005900,-0.001069,0.011902,0.008395,-0.009315,-0.004288,0.002730,-0.002904,...,-0.000429,-0.002427,-0.004951,-0.000071,0.003106,-0.000264,0.002794,-0.001990,0.000537,-0.002288
4,0.002359,0.002011,0.000671,0.005992,0.005120,0.005878,-0.004156,-0.004930,-0.007542,0.001422,...,0.006187,0.003312,-0.008371,0.002347,-0.001602,0.004468,-0.000183,-0.001356,0.006283,0.010786
11,-0.007630,0.006179,-0.008678,0.002850,-0.015907,-0.003556,0.012023,0.003135,-0.003019,0.016977,...,0.005002,0.004345,-0.001284,0.000843,0.006679,-0.002604,0.009731,0.005403,-0.003942,0.001610
17,0.000359,0.011913,-0.001688,0.003009,0.005049,-0.007076,-0.001782,-0.011211,-0.008768,0.001183,...,-0.002953,-0.002389,-0.009984,0.001183,0.000139,0.004810,0.000604,-0.004058,0.006402,0.010561
19,0.002182,-0.005412,0.001529,-0.000276,0.001899,0.005824,0.012396,-0.005323,0.004359,0.006758,...,0.006220,0.003243,0.000287,0.007862,-0.003352,-0.004294,0.004609,0.005015,0.002741,0.003260
27,-0.002894,0.005726,0.004879,0.000750,-0.002470,-0.005316,0.002477,-0.000922,-0.007154,0.004107,...,-0.001297,-0.001854,0.002073,-0.002201,0.003015,-0.002998,0.001753,-0.010310,-0.000759,0.001883
28,-0.007059,0.002486,-0.001655,-0.005595,0.003588,-0.000763,-0.004083,-0.007805,-0.005252,0.005013,...,-0.006312,-0.013393,0.000458,0.002910,-0.002428,0.001460,0.003668,0.001572,0.000475,-0.000668
39,0.009580,0.003498,0.005039,-0.000007,0.014106,-0.005306,0.002905,-0.007380,-0.000577,-0.000369,...,-0.000351,-0.008841,-0.006933,-0.000350,0.001729,0.010375,0.001885,-0.005291,-0.002005,0.008032
61,0.001480,0.002904,-0.007945,0.007666,0.003446,0.003241,-0.003079,-0.015888,0.001303,0.007713,...,0.004625,-0.005498,-0.011240,0.015706,0.007472,-0.003013,0.006068,0.005667,-0.005374,0.000423
